In [1]:
import numpy as np
from pyci.utils import AOint
from pyci.linalg import blas, lapack
molfile = '.xyz/He.xyz'

In [2]:
aoint = AOint('aug-cc-pvdz', molfile, wd='./', ncore=11, psi4mem='210 Gb', numpymem=210,
                         custom_basis=False,  psi4options={'puream': False})


In [8]:
import os

def orthonormalize(S):
    # check_overlap = np.allclose(np.diag(S), np.ones(S.shape[0]))
    # if not check_overlap:
    #     raise Exception('Improper overlap matrix was given.')
    svals, svecs = np.linalg.eigh(S)
    X = lapack.dinv(np.diag(np.sqrt(svals)))
    X = blas.dmul_mmm(svecs, X, svecs.T)
    S_p = blas.dmul_mmm(X.T, S, X)
    # Checking if orthonormalised
    check_overlap = np.allclose(S_p, np.eye(S.shape[0]), atol=1e-08)
    if not check_overlap:
        raise Exception("There's something wrong with the transformation.")
    return S_p, X


def SCF(AOint, scfoptions={}):
    options = {
        'maxiter': 75,
        'e_convergence': 1e-12,
        'd_convergence': 1e-10,
        'diis': True,
        'diis_iter': 15,
    }
    options.update(scfoptions)
    if not os.path.isfile(AOint.scratch+'ao_oeints.npz'):
        AOint.save_ao_oeints()
    if not os.path.isfile(AOint.scratch+'ao_erints.npz'):
        AOint.save_ao_erints()
    S, T, V = AOint.get_ao_oeints()
    I = AOint.get_ao_erints()
    H = T + V
    S_p, X = orthonormalize(S)
    F_p = blas.dmul_mmm(X.T, H, X)
    eps_p, C_p = np.linalg.eigh(F_p)
    C = blas.dmul_mm(X, C_p)
    nel = (AOint.wfn.nalpha() + AOint.wfn.nbeta())
    nmo = AOint.wfn.nmo()
    if nel % 2:
        raise Exception('Open-Shell system passed RHF algorithm.')
    ndocc = int(nel/2)
    options['nmo'] = nmo
    options['ndocc'] = ndocc
    C_occ = C[:, :ndocc]
    D = blas.dmul_mm(C_occ, C_occ.T)
    MAXITER = options.get('maxiter')
    DIIS_ITER = options.get('diis_iter') # int(N) if you want to switch of DIIS after 'N' scf cycles 
    E_CONV = options.get('e_convergence')
    D_CONV = options.get('e_convergence')
    E_old = 0.0
    D_old = np.zeros(D.shape)
    SCF_E= 0.0
    # begin Iterations
    print('==> Starting SCF Iterations <==\n')
    # Trial & Residual Vector Lists
    F_list = []
    DIIS_RESID = []
    # ==> SCF Iterations w/ DIIS <==
    for scf_iter in range(1, MAXITER + 1):
        # Build Fock matrix
        J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
        K = np.einsum('prqs,rs->pq', I, D, optimize=True)
        F = H + 2*J - K
        if scf_iter <= DIIS_ITER:
            diis_bool = True
            # Build DIIS Residual
            diis_r = X.dot(F.dot(D).dot(S) - S.dot(D).dot(F)).dot(X)
            # Append trial & residual vectors to lists
            F_list.append(F)
            DIIS_RESID.append(diis_r)
        else:
            diis_bool = False
        # Compute RHF energy
        SCF_E = np.einsum('pq,pq->', (H + F), D, optimize=True)
        dE = SCF_E - E_old
        dRMS = np.mean(diis_r**2)**0.5
        if diis_bool:
            diis_str = '   DIIS is ON'
        else:
            diis_str = '   DIIS is OFF'
        print(('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' %
            (scf_iter, SCF_E, dE, dRMS)) + diis_str)        
        # Checking if SCF Converged?
        # if (abs(dE) <= E_CONV):
        #     break
        # Always use Density convergence. 
        if np.allclose(D, D_old, atol=D_CONV):
            break
        E_old = SCF_E
        D_old = D
        if scf_iter >= 2:
            # Build B matrix
            B_dim = len(F_list) + 1
            B = np.empty((B_dim, B_dim))
            B[-1, :] = -1
            B[:, -1] = -1
            B[-1, -1] = 0
            for i in range(len(F_list)):
                for j in range(len(F_list)):
                    B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i],
                                        DIIS_RESID[j], optimize=True)
            # Build RHS of Pulay equation
            rhs = np.zeros((B_dim))
            rhs[-1] = -1
            # Solve Pulay equation for c_i's with NumPy
            coeff = lapack.dsolve(B, rhs)
            # Build DIIS Fock matrix
            F = np.zeros_like(F)
            for x in range(coeff.shape[0] - 1):
                F += coeff[x] * F_list[x]

        # Compute new orbital guess with DIIS Fock matrix
        F_p = X.dot(F).dot(X)
        e, C_p = np.linalg.eigh(F_p)
        C = X.dot(C_p)
        ndocc = options.get('ndocc')
        C_occ = C[:, :ndocc]
        D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
        # MAXITER exceeded?
        if (scf_iter == MAXITER):
            raise Exception("Maximum number of SCF iterations exceeded.")
    # Post iterations
    print('\nSCF converged.')
    print('Final RHF Energy: %.12f [Eh]' % SCF_E)
    return SCF_E, C


In [9]:
scf_e, C = SCF(aoint)

Size of the ERI tensor will be 0.00 Gb

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -2.7425529292621791 dE = -2.74255E+00 dRMS = 5.33318E-02   DIIS is ON
SCF Iteration   2: Energy = -2.8546341202662924 dE = -1.12081E-01 dRMS = 4.62690E-03   DIIS is ON
SCF Iteration   3: Energy = -2.8556953746537030 dE = -1.06125E-03 dRMS = 3.29923E-04   DIIS is ON
SCF Iteration   4: Energy = -2.8557046531332198 dE = -9.27848E-06 dRMS = 1.36690E-05   DIIS is ON
SCF Iteration   5: Energy = -2.8557046677103792 dE = -1.45772E-08 dRMS = 2.80185E-08   DIIS is ON
SCF Iteration   6: Energy = -2.8557046677075077 dE =  2.87148E-12 dRMS = 2.21924E-07   DIIS is ON
SCF Iteration   7: Energy = -2.8557046677104143 dE = -2.90656E-12 dRMS = 1.52191E-08   DIIS is ON
SCF Iteration   8: Energy = -2.8557046677104152 dE = -8.88178E-16 dRMS = 1.45117E-08   DIIS is ON

SCF converged.
Final RHF Energy: -2.855704667710 [Eh]


In [10]:
import psi4

In [11]:
psi4energy = psi4.energy('hf') 

In [12]:
pyci_e = scf_e + aoint.mol.nuclear_repulsion_energy()
print(pyci_e, psi4energy, abs(pyci_e- psi4energy))


-2.855704667710415 -2.855704667710431 1.5987211554602254e-14
